In [1]:
# https://stackoverflow.com/questions/45741878/using-binary-crossentropy-loss-in-keras-tensorflow-backend
# https://www.machinecurve.com/index.php/2019/10/22/how-to-use-binary-categorical-crossentropy-with-keras/

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.losses import BinaryCrossentropy
from sklearn.model_selection import train_test_split
from keras.callbacks import CSVLogger
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
physical_devices = tf.config.list_physical_devices('GPU') 
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)

%load_ext tensorboard
import datetime

!rm -rf ./logs/LSTM/

tf.config.experimental.list_physical_devices('GPU')

# RNN based Models (RNN & LSTM & GRU) using tensorflow keras

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
def convert_3D(matrix):
    return np.expand_dims(matrix, -1)

In [4]:
#retrieve and format data - into labels from the dataset
def labels(filename):
    data = np.matrix(pd.read_csv(filename).values)
    data_y = data[:, 0]
    data_y-=1 
    return data_y

In [5]:
with open("../archive/ProcessData.npy", 'rb') as f:
        training_data_x = np.load(f)
        test_x = np.load(f)
training_data_x = convert_3D(training_data_x) 
test_x = convert_3D(test_x) 

training_data_y = labels("../archive/exoTrain.csv")
test_y = labels("../archive/exoTest.csv")

training_data_x, validation_data_x, training_data_y, validation_data_y = train_test_split(training_data_x, training_data_y, test_size=0.3)

In [6]:
LSTMmodel = Sequential([
    LSTM(64, input_shape=(training_data_x.shape[1:]), return_sequences=True),
    #Dropout(0.3),
    LSTM(64),
    #Dropout(0.3),
    Dense(1, activation='sigmoid')
])

LSTMmodel.compile(
    loss=BinaryCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=0.000003),
    metrics=["accuracy"]
)

log_dir = "logs/LSTM/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

csv_logger = CSVLogger('keras-models/KerasLSTM/training.csv',separator=",", append=False)

LSTMmodel.fit(training_data_x, training_data_y, epochs=10, validation_data=(validation_data_x,validation_data_y), callbacks=[tensorboard_callback, csv_logger])

model = LSTMmodel
model.save('keras-models/KerasLSTM')

Epoch 1/10
112/112 [==============================] - 29s 228ms/step - loss: 0.6915 - accuracy: 0.9882 - val_loss: 0.6899 - val_accuracy: 0.9882
Epoch 2/10
112/112 [==============================] - 22s 201ms/step - loss: 0.6883 - accuracy: 0.9882 - val_loss: 0.6866 - val_accuracy: 0.9889
Epoch 3/10
112/112 [==============================] - 22s 199ms/step - loss: 0.6847 - accuracy: 0.9882 - val_loss: 0.6827 - val_accuracy: 0.9889
Epoch 4/10
112/112 [==============================] - 22s 199ms/step - loss: 0.6805 - accuracy: 0.9882 - val_loss: 0.6781 - val_accuracy: 0.9889
Epoch 5/10
112/112 [==============================] - 22s 200ms/step - loss: 0.6754 - accuracy: 0.9882 - val_loss: 0.6725 - val_accuracy: 0.9889
Epoch 6/10
112/112 [==============================] - 22s 201ms/step - loss: 0.6691 - accuracy: 0.9882 - val_loss: 0.6653 - val_accuracy: 0.9889
Epoch 7/10
112/112 [==============================] - 22s 201ms/step - loss: 0.6609 - accuracy: 0.9882 - val_loss: 0.6560 - val_ac

INFO:tensorflow:Assets written to: keras-models/KerasLSTM/assets


INFO:tensorflow:Assets written to: keras-models/KerasLSTM/assets


In [7]:
%tensorboard --logdir logs/LSTM/fit